In [64]:
%load_ext autoreload
%autoreload 2

import os, sys
import copy
import socket
from tqdm import tqdm
import torch
import pickle
from torch import optim
from pathlib import Path
from torch.utils.tensorboard import SummaryWriter

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "../../")))
from libs import fl, nn, agg, data, poison, log

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [65]:
# Save Logs To File (info | debug | warning | error | critical) [optional]
log.init("info")
#log.init("info", "federated.log")
#log.init("debug", "flkafka.log")

In [66]:
class FedArgs():
    def __init__(self):
        self.num_clients = 50
        self.epochs = 10
        self.local_rounds = 1
        self.client_batch_size = 32
        self.test_batch_size = 128
        self.learning_rate = 1e-4
        self.weight_decay = 1e-5
        self.cuda = False
        self.seed = 1
        self.tb = SummaryWriter('../../out/runs/federated/FLTrust', comment="Mnist Centralized Federated training")

fedargs = FedArgs()

In [67]:
use_cuda = fedargs.cuda and torch.cuda.is_available()
torch.manual_seed(fedargs.seed)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {"num_workers": 1, "pin_memory": True} if use_cuda else {}

In [68]:
host = socket.gethostname()
clients = [host + "(" + str(client + 1) + ")" for client in range(fedargs.num_clients)]

In [69]:
#Initialize Global and Client models
global_model = nn.ModelMNIST()
client_models = {client: copy.deepcopy(global_model) for client in clients}

# Function for training
def train_model(model, train_loader, fedargs, device):
    model, loss = fl.client_update(model,
                                train_loader,
                                fedargs.learning_rate,
                                fedargs.weight_decay,
                                fedargs.local_rounds,
                                device)
    return model, loss

In [70]:
# Load MNIST Data to clients
train_data, test_data = data.load_dataset("mnist")

In [71]:
# For securing if the next cell execution is skipped
FLTrust = None

<h1>FLTrust: Skip section below for any other averaging than FLTrust.</h1>

In [63]:
# For FLTrust
#############Skip this section for running other averaging
FLTrust = True
root_ratio = 0.01
train_data, root_data = torch.utils.data.random_split(train_data, [int(len(train_data) * (1-root_ratio)), 
                                                              int(len(train_data) * root_ratio)])
root_loader = torch.utils.data.DataLoader(root_data, batch_size=fedargs.client_batch_size, shuffle=True, **kwargs)

#global_model, _ = train_model(global_model, root_loader, fedargs, device)
#client_models = {client: copy.deepcopy(global_model) for client in clients}
#############

<h2>Resume</h2>

In [72]:
clients_data = data.split_data(train_data, clients)

<h1>Poison: Skip section below to run normal, modify if required.</h1>

In [39]:
# Poison a client
################Skip this section for running without poison
for client in range(10):
    clients_data[clients[client]] = poison.label_flip(clients_data[clients[client]], 4, 9, poison_percent = -1)
    
#clients_data[clients[0]] = poison.label_flip(clients_data[clients[0]], 6, 2, poison_percent = 1)
#clients_data[clients[0]] = poison.label_flip(clients_data[clients[0]], 3, 8, poison_percent = 1)
#clients_data[clients[0]] = poison.label_flip(clients_data[clients[0]], 1, 5, poison_percent = 1)

<h2>Resume</h2>

In [73]:
client_train_loaders, _ = data.load_client_data(clients_data, fedargs.client_batch_size, None, **kwargs)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=fedargs.test_batch_size, shuffle=True, **kwargs)

clients_info = {
        client: {"train_loader": client_train_loaders[client]}
        for client in clients
    }

In [74]:
import asyncio
import time

def background(f):
    def wrapped(*args, **kwargs):
        return asyncio.get_event_loop().run_in_executor(None, f, *args, **kwargs)

    return wrapped

@background
def process(client, epoch, model, train_loader, fedargs, device):
    # Train
    model, loss = train_model(model, train_loader, fedargs, device)

    #Plot and Log
    for local_epoch, loss in enumerate(list(loss.values())):
        fedargs.tb.add_scalars("Training Loss/" + client, {str(epoch): loss}, str(local_epoch + 1))

    log.jsondebug(loss, "Epoch {} of {} : Federated Training loss, Client {}".format(epoch, fedargs.epochs, client))
    log.modeldebug(model, "Epoch {} of {} : Client {} Update".format(epoch, fedargs.epochs, client))
    
    return model

In [ ]:
import time
start_time = time.time()
    
# Federated Training
for _epoch in tqdm(range(fedargs.epochs)):

    epoch = _epoch + 1
    log.info("Federated Training Epoch {} of {}".format(epoch, fedargs.epochs))

    # Gloabal Model Update
    if epoch > 1:
        # For Tmean, not impacts others as of now
        avgargs = {"beta": 10}
        
        # For FLTrust, if FLTrust section is skipped, this piece of code will be ignored automatically
        if FLTrust:
            global_model, _ = train_model(global_model, root_loader, fedargs, device)
        
        # Average
        global_model = fl.federated_avg(client_models, global_model, agg.Rule.FedAvg, **avgargs)
        log.modeldebug(global_model, "Epoch {} of {} : Server Update".format(epoch, fedargs.epochs))

        # Test
        global_test_output = fl.eval(global_model, test_loader, device, 4, 9)
        fedargs.tb.add_scalars("Test Evaluation", {
            'Gloabl Accuracy': global_test_output["accuracy"],
            'Attack Success Rate': global_test_output["attack"]["attack_success_rate"],
            'Misclassification Rate': global_test_output["attack"]["misclassification_rate"],
        }, epoch)
        log.jsoninfo(global_test_output, "Gloabl Test Outut after Epoch {} of {}".format(epoch, fedargs.epochs))
    
        # Update client models
        client_models = {client: copy.deepcopy(global_model) for client in clients}

    # Clients
    loop = asyncio.get_event_loop()
    tasks = [process(client, epoch, client_models[client],
                     clients_info[client]['train_loader'],
                     fedargs, device) for client in clients]
    updates = loop.run_until_complete(asyncio.gather(*tasks))
    client_models = {client: update for client, update in zip(clients, updates)}
    
print(time.time() - start_time)

 10%|█         | 1/10 [00:33<05:03, 33.72s/it]2021-08-17 11:03:49,147 - <ipython-input-75-c672cc171386>::<module>(l:8) : Federated Training Epoch 2 of 10 [MainProcess : MainThread (INFO)]
2021-08-17 11:03:55,519 - <ipython-input-75-c672cc171386>::<module>(l:30) : Gloabl Test Outut after Epoch 2 of 10 {
    "accuracy": 74.26,
    "attack": {
        "attack_success_count": 268,
        "attack_success_rate": 27.29124236252546,
        "instances": 982,
        "misclassification_rate": 34.72505091649695,
        "misclassifications": 341
    },
    "correct": 7426,
    "test_loss": 1.4976529468536377
} [MainProcess : MainThread (INFO)]
 20%|██        | 2/10 [01:16<05:10, 38.86s/it]2021-08-17 11:04:31,609 - <ipython-input-75-c672cc171386>::<module>(l:8) : Federated Training Epoch 3 of 10 [MainProcess : MainThread (INFO)]
2021-08-17 11:04:37,623 - <ipython-input-75-c672cc171386>::<module>(l:30) : Gloabl Test Outut after Epoch 3 of 10 {
    "accuracy": 84.96000000000001,
    "attack": {
  